In [1]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

# DAM

In [2]:
from fastai.vision.all import *
import pandas as pd
import cam
import util

In [3]:
base_model_name = 'densenet121-chexpert'

dls, labels = util.chexpert_data_loader(bs=16)

In [11]:
# Load the base model
model_base = cnn_learner(dls, densenet121, n_out=len(labels), path=Path('../saves/'), y_range=[0,1])
model_base.load(base_model_name)

# Create a new densenet121 model
model_dam = cnn_learner(dls, densenet121, n_out=len(labels), y_range=[0,1]).model

# Replace the body of new model with the body from the base model
base_model_body = model_base.model[0].state_dict()
model_dam[0].load_state_dict(base_model_body)

# Set all parameters to require grad
for param in model_dam.parameters():
    param.requires_grad = True
    
# Initialize DAM learner
dam_learner = util.DAM(model_dam, dls, folder='../saves/densenet121_256_32_hlcp_data_mod_w_img_augt_w_dam')

In [12]:
print(f'Initial AUC score: {dam_learner.eval()}')

Initial AUC score: 0.49975691311920867


In [13]:
%%notify
dam_learner.train(max_epoch=3, lr_div=1.5)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 7.94 GiB total capacity; 7.30 GiB already allocated; 18.00 MiB free; 7.38 GiB reserved in total by PyTorch)

<IPython.core.display.Javascript object>